## Build a News ETL Data Pipeline Using Python and SQLite

### Task 1: Import Libraries and Connect to the News API

In [22]:
import pandas as pd
import sqlite3
import logging
from newsapi import NewsApiClient
from datetime import datetime, timedelta, time
from airflow import DAG


In [23]:
news_api_key="8d9ffbb9befb4f70b3a012ff6e4e5cd6"

news_api=NewsApiClient(api_key=news_api_key)

### Task 2: Retrieve and Print News Articles

In [24]:
def extract_news_data():
    try:
        result = news_api.get_everything(q="AI", language="en",sort_by='publishedAt')
        logging.info("Connection is successful.")
        return result["articles"]
    except:
        logging.error("Connection is unsuccessful.")
        return None

articles = extract_news_data()

print(articles[:3])

[2024-06-29T17:40:00.209+0000] {535191998.py:4} INFO - Connection is successful.
[{'source': {'id': None, 'name': 'SiliconANGLE News'}, 'author': 'Robert Hof', 'title': 'Generative AI’s big copyright battle commences, and chip upstarts target AI workloads', 'description': 'It’s perhaps the biggest signal yet that generative artificial intelligence model providers aren’t going to get a free content ride: Record labels this week\xa0sued\xa0two AI music generators, and this one may prove to be the big battle over AI and copyright. Meant…', 'url': 'https://siliconangle.com/2024/06/28/generative-ais-big-copyright-battle-commences-chip-upstarts-target-ai-workloads/', 'urlToImage': 'https://d15shllkswkct0.cloudfront.net/wp-content/blogs.dir/1/files/2024/06/aimusicbattle-ideogram.png', 'publishedAt': '2024-06-28T17:37:31Z', 'content': 'It’s perhaps the biggest signal yet that generative artificial intelligence model providers aren’t going to get a free content ride: Record labels this week\xa0

### Task 3: Clean Author Column

In [25]:
def clean_author_column(text):
    try:
        return text.split(",")[0].title()
    except AttributeError:
        return "No Author"

### Task 4: Transform News Data

In [26]:
def transform_news_data(articles):
    article_list = []
    for i in articles:
        article_list.append([value.get("name", 0) if key == "source" else value for key, value in i.items() if key in ["author", "title", "publishedAt", "content", "url", "source"]])

    df = pd.DataFrame(article_list, columns=["Source", "Author Name", "News Title", "URL", "Date Published", "Content"])

    df["Date Published"] = pd.to_datetime(df["Date Published"]).dt.strftime('%Y-%m-%d %H:%M:%S')
    df["Author Name"] = df["Author Name"].apply(clean_author_column)
 
    return df

transformed_data = transform_news_data(articles)

print(transformed_data)

                 Source                            Author Name  \
0     SiliconANGLE News                             Robert Hof   
1             Unity.com                              No Author   
2           Hacker News                                Patrulo   
3             The Verge                             Jay Peters   
4                 Wired                           Tim Marchman   
..                  ...                                    ...   
95           Github.com                               Ozgrozer   
96                 Time                         Andrew R. Chow   
97        GlobeNewswire                              Aiconnect   
98    The Week Magazine  Theweek@Futurenet.Com (Justin Klawans   
99  The Washington Post                            Shira Ovide   

                                           News Title  \
0   Generative AI’s big copyright battle commences...   
1   185 Realistic Animated NPC Portraits Pack by A...   
2   Show HN: InlineGPT – prompt with

### Task 5: Load the Data into SQLite Database


In [27]:
def load_news_data(data):
    with sqlite3.connect("/usercode/news_data.sqlite") as connection:
        cursor = connection.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS news_table (
                "Source" VARCHAR(30),
                "Author Name" TEXT,
                "News Title" TEXT,
                "URL" TEXT,
                "Date Published" TEXT,
                "Content" TEXT
            )
        ''')
    data.to_sql(name="news_table", con=connection, index=False, if_exists="append")
 
load_news_data(transformed_data)

### Task 8: Verify Data Loading

In [28]:

to_date=datetime.utcnow().date()
from_date=to_date-timedelta(days=1)
dag=DAG(dag_id="news_etl", default_args={'start_date': datetime.combine(from_date, time(0,0)), 'retries':1},
schedule_interval='@daily',)




/tmp/ipykernel_363/2446328449.py:3 RemovedInAirflow3Warning: Param `schedule_interval` is deprecated and will be removed in a future release. Please use `schedule` instead.

In [29]:
with sqlite3.connect("/usercode/etl_news_data.sqlite") as connection:
    df = pd.read_sql("SELECT * FROM news_table;", connection)
df.head()

,Source,Author Name,News Title,URL,Date Published,Content
0,Yahoo Entertainment,Mariella Moon,Amazon investigating Perplexity AI after accus...,https://consent.yahoo.com/v2/collectConsent?se...,2024-06-28 13:30:03,"If you click 'Accept all', we and our partners..."
1,CNET,Katelyn Chedraoui,Best AI Image Generators of 2024,https://www.cnet.com/tech/services-and-softwar...,2024-06-28 12:31:00,As artificial intelligence technology advances...
2,CNET,Gael Cooper,Toys 'R' Us Kids Meet AI in a Video Using Sora...,https://www.cnet.com/tech/services-and-softwar...,2024-06-28 03:40:00,"Toys ""R"" Us released a video made with a new a..."
3,The Next Web,The Conversation,Does your service business need AI? Here are 4...,https://thenextweb.com/news/does-your-service-...,2024-06-28 08:30:19,Artificial intelligence is the big thing right...
4,ReadWrite,Graeme Hanna,YouTube wants record labels to license music f...,https://readwrite.com/youtube-wants-record-lab...,2024-06-28 15:27:05,YouTube is reportedly locked in talks with maj...
